In [1]:
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.fsl.utils import Merge, ImageMeants, Split
from nipype.interfaces.fsl.model import Randomise, Cluster
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.fsl.maths import ApplyMask, Threshold
from pandas import DataFrame, Series

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

#other study-specific variables
project_home = '/Users/catcamacho/Box/SNAP/BABIES'
preproc_dir = project_home + '/proc/asl_preproc'
output_dir = project_home + '/proc/asl_lme_group'
wkflow_dir = project_home + '/workflows'
template = project_home + '/templates/T2w_BABIES_template_2mm.nii.gz'
mask = project_home + '/templates/BABIES_gm_mask_2mm.nii.gz'
subject_info = project_home + '/misc/subjects.csv'

# Group analysis models (predicting FC)
models = ['brain ~ age_wks_MC + male + cesd_MC + Negative_affect + cesd_MC*Negative_affect', 
          'brain ~ age_wks_MC + male + cesd_MC + Negative_affect + cesd_MC*Negative_affect + mom_age_MC']

model_names = ['age_sex_only', 'all_covariates']

terms = ['cesd_MC', 'Negative_affect', 'cesd_MC_Negative_affect']

# for GM mask, # voxels = 98764
# for brainmask, # voxels = 113120

In [2]:
# Grab the subject cbf maps 
grabcbfdata = Node(DataGrabber(template=preproc_dir + '/cbf_volume/*/salsop_cbf_masked.nii.gz', 
                               sort_filelist=True, 
                               outfields=['cbf_list']), 
                   name='grabcbf')

# Sink relavent data
substitutions = [('brain~ageMC+sex+cesd_MC+Negative_affect+cesd_MC*Negative_affect','age_sex_only'),
                 ('brain~ageMC+sex+cesd_MC+Negative_affect+cesd_MC*Negative_affect+mom_age_MC+breastfed+MDD_hist','all_covariates')]
datasink = Node(DataSink(substitutions=substitutions, 
                         base_directory=output_dir,
                         container=output_dir), 
                name='datasink')

In [10]:
# LMEM for MRI data (3D nifti data)
def mri_lmem(model, mask, subject_dataframe, subject_files, grouping_variable):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)

    from os import getcwd
    from os.path import abspath
    import statsmodels.formula.api as smf
    from nibabel import load, save, Nifti1Image
    from numpy import array, empty_like, stack, nditer, zeros_like, zeros, sum
    from pandas import DataFrame, read_csv, Series, concat
    from warnings import filterwarnings
    filterwarnings('ignore')

    working_dir = getcwd() + '/'
    subj_data = read_csv(subject_dataframe, header=0, index_col=None)

    # Load the brain data
    brain_niftis = load(subject_files)
    brain_data_4D = brain_niftis.get_data()

    # Load the mask
    mask_nifti = load(mask)
    mask = mask_nifti.get_data()
    num_voxels = sum(mask)

    ## Preallocate the output arrays
    # for the model
    BIC_data = zeros_like(mask).astype(float)
    AIC_data = zeros_like(mask).astype(float)
    pval_intercept_data = zeros_like(mask).astype(float)
    pval_cesd_data = zeros_like(mask).astype(float)
    pval_neg_data = zeros_like(mask).astype(float)
    pval_cesd_neg_data = zeros_like(mask).astype(float)
    # per subject
    residuals_data = zeros_like(brain_data_4D).astype(float)
    pred_values_data = zeros_like(brain_data_4D).astype(float)

    # Set up the actual loops to pull in subject data and do the modeling
    for x in range(0,mask.shape[0]):
        for y in range(0,mask.shape[1]):
            for z in range(0,mask.shape[2]):
                if mask[x][y][z] == 1:
                    voxel = zeros(brain_data_4D.shape[3])
                    for a in range(0,brain_data_4D.shape[3]):
                        voxel[a] = brain_data_4D[x][y][z][a]
                    voxel = Series(voxel, index=subj_data.index, name='brain')
                    data = concat([voxel, subj_data],axis=1)
                    
                    mlm = smf.mixedlm(model, data, groups=data[grouping_variable])
                    mod = mlm.fit()
                    pval_intercept_data[x][y][z] = 1 - (mod.pvalues[0])
                    pval_cesd_data[x][y][z] = 1 - (mod.pvalues[3])
                    pval_neg_data[x][y][z] = 1 - (mod.pvalues[4])
                    pval_cesd_neg_data[x][y][z] = 1 - (mod.pvalues[5])
                    BIC_data[x][y][z] = mod.bic
                    AIC_data[x][y][z] = mod.aic
                    residuals = mod.resid
                    pred_values = Series(mod.predict(), index = subj_data.index)
                    
                    for d in range(0,brain_data_4D.shape[3]):
                        residuals_data[x][y][z][d] = residuals.tolist()[d]
                        pred_values_data[x][y][z][d] = pred_values.tolist()[d]

    # Save the ouputs as nifti files
    output_data = [BIC_data, AIC_data, pval_intercept_data, pval_cesd_data,
                    pval_neg_data, pval_cesd_neg_data, residuals_data, 
                    pred_values_data]
    output_niftis = [Nifti1Image(result, mask_nifti.affine) for result in output_data]
    
    output_filenames = ['BICs.nii.gz','AICs.nii.gz','pval_intercept_data.nii.gz',
                        'pval_cesd_data.nii.gz','pval_neg_data.nii.gz',
                        'pval_cesd_neg_data.nii.gz','residuals_data.nii.gz',
                        'pred_values_data.nii.gz']
    for e in range(0,len(output_niftis)):
        save(output_niftis[e], working_dir + output_filenames[e])
    
    output_volumes = [abspath(output_filenames[0]),
                      abspath(output_filenames[1]),
                      abspath(output_filenames[2]), 
                      abspath(output_filenames[3]), 
                      abspath(output_filenames[4]), 
                      abspath(output_filenames[5]), 
                      abspath(output_filenames[6]), 
                      abspath(output_filenames[7])]
    
    return(output_volumes)

In [11]:
## Analysis nodes

#Merge subject files together
merge = Node(Merge(dimension='t'), name='merge')

apply_mask = Node(ApplyMask(mask_file=mask, nan2zeros=True), name='apply_mask')

# Linear mixed effects modeling
lmemodel = Node(Function(input_names = ['model', 'mask', 'subject_dataframe', 
                                        'subject_files', 'grouping_variable'], 
                         output_names = ['output_volumes'], 
                         function=mri_lmem), 
                name='lmemodel')
lmemodel.iterables = [('model', models)]
lmemodel.inputs.mask = mask
lmemodel.inputs.subject_dataframe = subject_info
lmemodel.inputs.grouping_variable = 'id'

# Mask the file to only significant voxels for clustering
mask_stat = Node(Binarize(), name = 'mask_stat')

# Cluster the results
cluster_results = MapNode(Cluster(threshold=0.95,
                                  out_index_file=True,
                                  out_localmax_txt_file=True),
                          name='cluster_results', 
                          iterfield = ['in_file'])

In [ ]:
LMEManalysisflow = Workflow(name='LMEManalysisflow')
LMEManalysisflow.connect([(grabcbfdata, merge,[('cbf_list', 'in_files')]),
                          (merge, apply_mask, [('merged_file','in_file')]),
                          (apply_mask, lmemodel, [('out_file','subject_files')]),
                          (apply_mask, datasink, [('out_file','merged_subj_betas')]),
                          (lmemodel, datasink, [('output_volumes','output_volumes')])
                         ])
LMEManalysisflow.base_dir = wkflow_dir
LMEManalysisflow.write_graph(graph2use='flat')
LMEManalysisflow.run('MultiProc', plugin_args={'n_procs':2})